In [0]:
!wget -q https://www.dropbox.com/s/yigijs122togfk4/embeddings.dat?dl=1  -O embeddings.dat
!wget -q https://www.dropbox.com/s/qdhtexle4p0ngc3/DatasetSplitCSV.zip?dl=1 -O DatasetSplitCSV.zip
!unzip -q DatasetSplitCSV.zip

In [0]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [0]:
#Parameters
params={}
params["embeddingType"]="raw" # post
params["mapReduceFunc"]="Pad" # Avg

In [0]:
def mapReduce(embed,funcName):
    if funcName=="Avg":
        embed=np.average(embed,axis=0)
    if funcName=="Pad":
        embed=np.pad(embed, [( 0,6-embed.shape[0]), (0, 0)], mode='constant', constant_values=0)
        embed=embed.reshape(-1)
    return embed

In [0]:
train=pd.read_csv("train.csv",header=None)
valid=pd.read_csv("valid.csv",header=None)
test=pd.read_csv("test.csv",header=None)
embeddings=np.load("embeddings.dat",allow_pickle=True)

In [8]:
embeddings["post"]['03-02-04-01-02-02-02.wav'].dtype

dtype('uint8')

In [9]:
list(embeddings['raw'].keys())[2000]

'03-02-04-01-02-02-02.wav'

In [0]:
# test=[]
# for x in train[0]:
#     embed=embeddings[params["embeddingType"]][x]

#     test.append(embed)

In [0]:
# np.array(test).shape

In [12]:
# test[0].s

AttributeError: ignored

In [0]:
# padded_inputs[0]

In [0]:
x_train=[]
x_val=[]
x_test=[]
for x in train[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_train.append(mapReduce(embed,params["mapReduceFunc"]))

for x in valid[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_val.append(mapReduce(embed,params["mapReduceFunc"]))

for x in test[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_test.append(mapReduce(embed,params["mapReduceFunc"]))

x_train=np.array(x_train)
x_val=np.array(x_val)
x_test=np.array(x_test)

y_train=np.array(train[1].astype('category').cat.codes)
y_val=np.array(valid[1].astype('category').cat.codes)
y_test=np.array(test[1].astype('category').cat.codes)



In [0]:
# from collections import Counter
# shapes=[]
# for f,embed in embeddings['raw'].items():
#     shapes.append((embed.shape))
# Counter(shapes)

Average Embeddings

In [0]:
inputs = keras.Input(shape=(x_train.shape[-1],), name="log-mel")
# x = tf.keras.layers.Reshape((6, 128), input_shape=(x_train.shape[-1],))(inputs)
# x = layers.LSTM(128,input_shape=((6,128)))(x)
x = layers.Dense(64, activation="tanh", name="dense_1")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="tanh", name="dense_2")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32, activation="tanh", name="dense_3")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(8, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [0]:
# inputs = keras.Input(shape=(x_train.shape[-1],), name="log-mel")
# x = tf.keras.layers.Reshape((6, 128), input_shape=(x_train.shape[-1],))(inputs)
# x = layers.LSTM(128,input_shape=((6,128)))(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(64, activation="tanh", name="dense_1")(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(32, activation="tanh", name="dense_2")(x)
# x = layers.Dropout(0.5)(x)
# outputs = layers.Dense(8, activation="softmax", name="predictions")(x)

# model = keras.Model(inputs=inputs, outputs=outputs)

In [0]:
    # model.add(layers.LSTM(128, input_shape=(input_shape[0], input_shape[1])))
    # model.add(layers.BatchNormalization())
    # model.add(layers.Dropout(0.5))
    # model.add(layers.Dense(32, activation='relu'))
    # model.add(layers.Dense(16, activation='tanh'))
    # model.add(layers.Dense(num_classes, activation='softmax'))

In [33]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.sparse_categorical_accuracy])
print('# Fit model on training data')
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=200,
                    validation_data=(x_val, y_val))

print('\nhistory dict:', history.history)

# Fit model on training data
Epoch 1/200
23/23 [==============================] - 0s 8ms/step - loss: 2.0730 - sparse_categorical_accuracy: 0.1415 - val_loss: 2.0417 - val_sparse_categorical_accuracy: 0.3035
Epoch 2/200
23/23 [==============================] - 0s 4ms/step - loss: 2.0363 - sparse_categorical_accuracy: 0.2136 - val_loss: 1.9938 - val_sparse_categorical_accuracy: 0.2994
Epoch 3/200
23/23 [==============================] - 0s 4ms/step - loss: 2.0164 - sparse_categorical_accuracy: 0.2442 - val_loss: 1.9568 - val_sparse_categorical_accuracy: 0.3136
Epoch 4/200
23/23 [==============================] - 0s 4ms/step - loss: 1.9800 - sparse_categorical_accuracy: 0.2925 - val_loss: 1.9345 - val_sparse_categorical_accuracy: 0.3259
Epoch 5/200
23/23 [==============================] - 0s 5ms/step - loss: 1.9601 - sparse_categorical_accuracy: 0.3170 - val_loss: 1.9199 - val_sparse_categorical_accuracy: 0.3360
Epoch 6/200
23/23 [==============================] - 0s 4ms/step - loss: 1.9

In [65]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)


# Evaluate on test data
4/4 [==============================] - 0s 2ms/step - loss: 1.9889 - sparse_categorical_accuracy: 0.2648
test loss, test acc: [1.9889146089553833, 0.26476576924324036]
